In [132]:
import re
from tqdm import tqdm_notebook
from nltk.corpus import stopwords
from tensorflow.keras import regularizers, initializers, optimizers, callbacks
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.utils.np_utils import to_categorical
from tensorflow.keras.layers import *
from tensorflow.keras.models import Sequential
import pandas as pd
import numpy as np
import os

In [3]:
test=pd.read_csv(r"C:\Users\THAKUR ABHINAV\Desktop\test.csv")
train=pd.read_csv(r"C:\Users\THAKUR ABHINAV\Desktop\train.csv")
test.shape,train.shape

((5200, 4), (20800, 5))

In [4]:
x_train=train["title"]+" "+train["text"]
y_train=train.label
x_train.shape,y_train.shape


((20800,), (20800,))

In [5]:
y = pd.get_dummies(y_train)
y = np.array(y)

In [6]:
import nltk
from nltk.corpus import stopwords
stop=stopwords.words("english")
len(stop)

179

In [8]:

def clean_text(text):
    output=""
    text=str(text).replace(r'http[\w:/\.]+','') # removing urls
    text=str(text).replace(r'[^\.\w\s]','') #remove everything but characters and punctuation 
    text=str(text).replace(r'\.\.+',' ') #replace multiple periods with a single one 
    text=str(text).replace(r'\.',' . ') #replace periods with a single one 
    text=str(text).replace(r'\s\s+',' ') #replace multiple white space with a single one 
    text=str(text).replace("\n", "") #removing line breaks
    text=re.sub(r'[^\w\s]','',text).lower() #to upper to lower
    text=text.split(" ")
    for word in text:
        if word not in stop:
            output=output+" "+word
    return str(output.strip())[0:].replace(" "," ")

In [16]:
texts=[]
for line in tqdm_notebook(x_train,total=train.shape[0]):
    texts.append(clean_text(line))

C:\Users\THAKUR ABHINAV\anaconda3\lib\site-packages\ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [17]:
len(texts)

20800

In [18]:
def save_file(filename,text):
    lines=list()
    for l in text:
        lines.append(l)
    data="\n".join(lines)
    file = open(filename, "w+", encoding="utf-8")
    file.write(data)
    file.close()

In [36]:
save_file("ram.txt",texts)

In [19]:
def load_txt_file(filename):
    file=open(filename,'r+',encoding="utf-8")
    text=file.read()
    file.close()
    return text

In [12]:
abc=load_txt_file("ram.txt")


In [13]:
textz=abc.split('\n')
len(textz)

20800

In [14]:
x_test=test["title"]+" "+test["text"]
len(x_test)

5200

In [15]:
test.shape

(5200, 4)

In [18]:
text_test=[]
for line in tqdm_notebook(x_test,total=test.shape[0]):
    text_test.append(clean_text(line))

C:\Users\THAKUR ABHINAV\anaconda3\lib\site-packages\ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [22]:
len(text_test)

5200

In [20]:
MAX_NB_WORDS=100000

In [21]:
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
word_index = tokenizer.word_index
print('Vocabulary size:', len(word_index))

Vocabulary size: 226684


In [22]:
#padding

data = pad_sequences(sequences, padding = 'post', maxlen = 1000) #post means 0 at last
print('Shape of data tensor:', data.shape)

Shape of data tensor: (20800, 1000)


In [23]:
from sklearn.model_selection import train_test_split

In [24]:
x_train1,x_val,y_train1,y_val=train_test_split(data,y,test_size=0.2,random_state=42)

In [25]:
x_train1.shape,y_train1.shape,x_val.shape,y_val.shape

((16640, 1000), (16640, 2), (4160, 1000), (4160, 2))

In [108]:
len(word_index)

226684

In [27]:
#embeddings

embedding_index={}
file=open("glove100.txt",encoding="utf-8")
for line in file:
    values=line.split()
    word=values[0]
    embedding_index[word]=np.asarray(values[1:],dtype='float32')
file.close()    

In [34]:
EMBEDDING_DIM = 100 # embedding dimensions for word vectors
embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
for word,i in word_index.items():
    embedding_vector=embedding_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i]=embedding_vector

In [107]:
embedding_matrix.shape

(226685, 100)

In [50]:
'''input_x = Input(shape=(None,))
x=Embedding(len(word_index) + 1,
  EMBEDDING_DIM,
  weights = [embedding_matrix],
  input_length = 1000,
  trainable=False,
  name = 'embeddings')(input_x)'''

In [63]:
#x1=Convolution1D(filters=128, kernel_size=3,activation='relu')(x)
#x1_2=MaxPool1D(pool_size=5)(x1)

In [56]:
#x2=Convolution1D(filters=128, kernel_size=4,activation='relu')(x)
#x2_2=MaxPool1D(pool_size=5)(x2)

In [62]:
#x3=Convolution1D(filters=128, kernel_size=5,activation='relu')(x)
#x3_2=MaxPool1D(pool_size=5)(x3)

In [65]:
#x4=concatenate([x1_2,x2_2,x3_2],axis = -1)

In [67]:
#x5=Convolution1D(filters=128, kernel_size=5,activation='relu')(x4)
#x5_2=MaxPool1D(pool_size=5)(x5)

In [74]:
#x6=Convolution1D(filters=128, kernel_size=5,activation='relu')(x5_2)
#x6_2=MaxPool1D(pool_size=5)(x6)


TensorShape([Dimension(None), Dimension(None), Dimension(128)])

In [73]:
#x7=Flatten()(x6_2)

TensorShape([Dimension(None), Dimension(None)])

In [89]:
#x8=Dense(128,activation='relu')(x7)
#x8_2=Dropout(rate=0.2)(x8)
MAX_SEQUENCE_LENGTH = 1000

In [140]:
model = Sequential()
#model.add(Input(shape=(MAX_SEQUENCE_LENGTH), dtype='int32'))
model.add(Embedding(len(word_index) + 1,EMBEDDING_DIM,weights = [embedding_matrix],input_length = MAX_SEQUENCE_LENGTH,trainable=False,
 name = 'embeddings',input_shape=(1000,)))
model.add(LSTM(60, return_sequences=True,name='lstm_layer'))
model.add(GlobalMaxPool1D())
model.add(Dropout(0.1))
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(2, activation='sigmoid'))

In [142]:
model.compile(optimizer='adam', loss='binary_crossentropy',metrics = ['accuracy'])

In [145]:
#import tensorflow as tf
#with tf.device("/device:GPU:0"):
 #   model.fit(x_train1, y_train1, epochs = 10, batch_size=128, validation_data=(x_val, y_val))